In [5]:
from datasets import load_dataset, load_from_disk
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import soundfile as sf
import torch
from jiwer import wer

In [6]:
# librispeech_eval = load_dataset("/home/chenhonghua/Project/Speech/librispeech_asr/librispeech_asr.py", "clean", split="test", cache_dir='/data/chenhonghua/datasets')
librispeech_eval = load_from_disk("/data/chenhonghua/datasets/librispeech_asr_dummy")

In [8]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large").to("cuda")
processor = WhisperProcessor.from_pretrained("openai/whisper-large")

In [13]:
librispeech_eval

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [15]:
def map_to_pred(batch):
    input_features = processor(batch["audio"]["array"], return_tensors="pt").input_features

    with torch.no_grad():
        logits = model(input_features.to("cuda")).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids, normalize = True)
    batch['text'] = processor.tokenizer._normalize(batch['text'])
    batch["transcription"] = transcription
    return batch

result = librispeech_eval.map(map_to_pred, batched=True, batch_size=1) #, remove_columns=["speech"])

print("WER:", wer(result["text"], result["transcription"]))

  0%|          | 0/73 [00:00<?, ?ba/s]

TypeError: list indices must be integers or slices, not str

In [9]:
librispeech_eval

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})